# Rev.ai Microphone Stream output
Writing the output directly to the webcam then writing to the HDMI output.
Note, for testing purposes alone so haven't included a script for recorded output text.
## Importing Relavent Modules

In [1]:
import threading
import numpy as np
import time
import ipywidgets as ipw
from matplotlib import pyplot as plt
import queue
import cv2
import os

from rev_ai.models import MediaConfig
from rev_ai.streamingclient import RevAiStreamingClient
from PPFunctions_Live import *
from time import sleep

## Hardware Constraints for Board:

In [2]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")

pAudio = base.audio
pAudio.set_volume(20)
pAudio.select_microphone() # using AUX cable connected to Headphones as microphone input

# Applying OpenCV filters on Webcam input
### 1. Initialize HDMI I/O

In [3]:
# monitor configuration: 640*480 @ 60Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

In [ ]:
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

### Step 2: Initialize camera from OpenCV

In [ ]:
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))

### Function to Display Frame:

In [ ]:
def video_output(sub, videoIn):
    ret, frame_vga = videoIn.read()

    if (ret):
        outframe = hdmi_out.newframe()
        outframe[:] = frame_vga

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8

        # area of interest, where subtitles are placed:
        tr = int(frame_in_w - 10), int(frame_in_h - 34)
        bl = 10, int(frame_in_h - 3)

        x = int(10)
        y = int(frame_in_h - 10)

        img = cv2.rectangle(outframe, tr, bl, (0, 0, 0), -1)
        image = cv2.putText(img, sub, (x, y), font, font_scale, (255, 255, 255), 2, cv2.LINE_AA)

        hdmi_out.writeframe(img)
    else:
        raise RuntimeError("Error while reading from camera.")

In [ ]:
class VideoThreading():
    def __init__(self, start=True):
        self.stopping = True
        self.sub = ""
        
    def _do(self):
        while not self.stopping:
            video_output(self.sub, videoIn)
            sleep(0.125)
        
    def start(self):
        if self.stopping:           
            self.stopping = False
            thread = threading.Thread(target=self._do)
            thread.start()
        
    def stop(self):
        self.stopping = True
        

# Defining Microphone Stream
Threaded class used to process audio into subtitles in real-time. Code based upon example from Strath-SDR RFSoC QPSK and rev.ai sample python code.
## Audio Thread:

In [ ]:
class AudioThreading():
    def __init__(self, pynq_audio, record_time, chunk, start=True, src=0):
        """
        Create new dma-based data timer.
        callback: function to call with data chunk
        gen: function to call to return data chunk
             (usually a dma channel's transfer function)
        """
        self.stopping = not start
        self.record_time = record_time
        self._buffer = queue.Queue()
        self._pynq_audio = pynq_audio
        self.chunk = chunk
        self.closed = True
        
        self.stopped = False

    def _do(self):
        """
        Generate new data and restart timer thread.
        Should never be run directly. use `start()` instead.
        """
        while not self.stopping:
            self._pynq_audio.record(self.record_time)
            self._buffer.put(self._pynq_audio.buffer << 8)

    def __enter__(self):
        
        self._audio_interface = self._pynq_audio
        
        """ Start the data generator thread. """
        if not self.stopping:           
            self.stopping = False
            thread = threading.Thread(target=self._do)
            thread.start()
            
        self.closed = False
        return self
    
    def __exit__(self, type, value, traceback):
        """
        Stop a running data generator thread.
        Does not need a lock, since the spawned timer thread will only read `self.stopping`.
        """
        self.stopped = True
        self.closed = True
        self._buffer.put(None)
    
    # Rev.ai Code example for microphone stream:
    def generator(self):
        while not self.stopping:
            """
            Use a blocking get() to ensure there's at least one chunk of
            data, and stop iteration if the chunk is None => stop on pause!
            """
            chunk = self._buffer.get()
            if chunk is None:
                return
            data = [chunk]

            while True:
                try:
                    chunk = self._buffer.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b''.join(data)

## Personal Access Token for Rev.ai:

In [ ]:
access_token = "020xN8wEvpFJ57K5xz4CjnhKVkC0kDJO74fKvx58chPRJQHUChMfrQyTWWooMnfO5H5kyGiVdHJHSroppSQFIU9g69v2E"

## Routing Audio Through Logic Fabric:
Sampling rate of Audio Codec

In [ ]:
rate = 48 * 1000 # Technically, codec is 96 kHz, however two channels will be used
chunk = int(rate/20000) # Approx 4s

## Raw Microphone Input to Create Media Config:
96 kHz interleaved, signed 32-bit, dual channel audio

In [ ]:
example_mc = MediaConfig('audio/x-raw', 'interleaved', rate, 'S32LE', 2) 
streamclient = RevAiStreamingClient(access_token, example_mc)

## Starting Video Thread:

In [ ]:
video_thread = VideoThreading()
video_thread.start()

## Opens microphone input, input will stop after a keyboard interrupt:

In [ ]:
sub_prev = ''

with AudioThreading(pAudio, 1, chunk) as stream:

    try:
        ## Starts the server connection and thread sending microphone audio
        print("new chunk!")
        response_gen = streamclient.start(stream.generator())
        ## Iterates through responses and prints them
        for response in response_gen:
            sub = real_t(response)
            if sub != sub_prev:
                sub_prev = sub
                video_thread.sub = sub_prev

    except KeyboardInterrupt:
        stream.stopping = True
        ## Ends the WebSocket connection.
        streamclient.end()
        pass

## Closing Webcam Connection and HDMI Output:

In [ ]:
videoIn.release()
hdmi_out.stop()
del hdmi_out